In [ ]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))

In [ ]:
set.seed(42)

In [ ]:
proj <- loadArchRProject("../data/snATAC", showLogo = FALSE)

In [ ]:
table(proj@cellColData$Sample)

In [ ]:
df_sample_annotation <- read.csv("../../../snATAC/metadata/sample_annotation.csv", header = TRUE)
head(df_sample_annotation)

sample_to_region <- df_sample_annotation$region_novel
sample_to_patient <- df_sample_annotation$patient
sample_to_patient_region <- df_sample_annotation$patient_region_id
sample_to_patient_group <- df_sample_annotation$patient_group

names(sample_to_region) <- df_sample_annotation$sample_id
names(sample_to_patient) <- df_sample_annotation$sample_id
names(sample_to_patient_region) <- df_sample_annotation$sample_id
names(sample_to_patient_group) <- df_sample_annotation$sample_id

In [ ]:
proj@cellColData$region <- stringr::str_replace_all(proj@cellColData$Sample, sample_to_region)
proj@cellColData$patient <- stringr::str_replace_all(proj@cellColData$Sample, sample_to_patient)
proj@cellColData$patient_region_id <- stringr::str_replace_all(proj@cellColData$Sample, sample_to_patient_region)
proj@cellColData$patient_group <- stringr::str_replace_all(proj@cellColData$Sample, sample_to_patient_group)

In [ ]:
proj <- addIterativeLSI(
    ArchRProj = proj,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI", 
    iterations = 3, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.2), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30,
    force = TRUE
)

In [ ]:
proj <- addUMAP(
    ArchRProj = proj, 
    reducedDims = "IterativeLSI", 
    name = "UMAP", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine",
    verbose = FALSE,
    force = TRUE
)

In [ ]:
p1 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "patient", embedding = "UMAP",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

p2 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "region", embedding = "UMAP",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

p3 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "patient_group", embedding = "UMAP",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

options(repr.plot.width = 20, repr.plot.height = 6)

p1 + p2 + p3

In [ ]:
proj <- addHarmony(
    ArchRProj = proj,
    reducedDims = "IterativeLSI",
    name = "Harmony",
    groupBy = c("patient", "Sample", "region"),
    force = TRUE
)

proj <- addUMAP(
    ArchRProj = proj, 
    reducedDims = "Harmony", 
    name = "UMAP_Harmony", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine",
    verbose = FALSE,
    force = TRUE
)

In [ ]:
p1 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "patient", embedding = "UMAP_Harmony",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

p2 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "region", embedding = "UMAP_Harmony",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

p3 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "patient_group", embedding = "UMAP_Harmony",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

options(repr.plot.width = 20, repr.plot.height = 6)

p1 + p2 + p3

In [ ]:
proj <- addImputeWeights(proj, reducedDims = "Harmony")

In [ ]:
p1 <- plotEmbedding(ArchRProj = proj,
                    colorBy = "GeneScoreMatrix",
                    name = "POSTN",
                    embedding = "UMAP_Harmony",
                    plotAs = "points",
                   size = 1) +
    theme_cowplot() +
    xlab("UMAP1") + ylab("UMAP2") +
    ggtitle("POSTN") +
    theme(legend.title = element_blank())

p2 <- plotEmbedding(ArchRProj = proj,
                    colorBy = "GeneScoreMatrix",
                    name = "NPR3",
                    embedding = "UMAP_Harmony",
                    plotAs = "points",
                   size = 1) +
    theme_cowplot() +
        xlab("UMAP1") + ylab("UMAP2") +
    ggtitle("NPR3") +
        theme(legend.title = element_blank())

p3 <- plotEmbedding(ArchRProj = proj,
                    colorBy = "GeneScoreMatrix",
                    name = "CDH11",
                    embedding = "UMAP_Harmony",
                    plotAs = "points",
                   size = 1) +
    theme_cowplot() +
        xlab("UMAP1") + ylab("UMAP2") +
    ggtitle("CDH11") +
        theme(legend.title = element_blank())


options(repr.plot.height = 5, repr.plot.width = 15)

p1 + p2 + p3

In [ ]:
## clustering
proj <- addClusters(
    input = proj,
    reducedDims = "Harmony",
    method = "Seurat",
    name = "Clusters",
    resolution = 0.4,
    force = TRUE
)

p1 <- plotEmbedding(ArchRProj = proj, 
              colorBy = "cellColData", 
              name = "Clusters", embedding = "UMAP_Harmony",plotAs = "points",
                   size = 0.5, labelAsFactors = FALSE, rastr = FALSE) +
    theme_cowplot()

options(repr.plot.height = 6, repr.plot.width = 6)

p1

In [ ]:
table(proj@cellColData$Clusters)

In [ ]:
cM <- confusionMatrix(paste0(proj$Clusters), paste0(proj$Sample))
cM

In [ ]:
# options(repr.plot.width = 12, repr.plot.height = 3)

# library(pheatmap)
# cM <- cM / Matrix::rowSums(cM)
# p <- pheatmap::pheatmap(
#     mat = as.matrix(cM), 
#     color = paletteContinuous("whiteBlue"), 
#     border_color = "black"
# )
# p

In [ ]:
meta.data <- as.data.frame(proj@cellColData)

In [ ]:
cols <- ArchR::paletteDiscrete(meta.data$Clusters)

In [ ]:
df_plot <- meta.data %>%
    group_by(patient, Clusters) %>%
    summarise(counts = n()) %>%
    mutate(cell_proportion = counts / sum(counts))

p1 <- ggplot(df_plot, aes(patient, cell_proportion, fill=Clusters)) + 
    geom_bar(stat="identity", position = position_stack(reverse = TRUE)) +
    scale_fill_manual(values = cols) +
    theme_cowplot() +
    xlab("") + ylab("") +
    theme(axis.text.x = element_text(angle = 60, hjust = 1))


options(repr.plot.width = 10, repr.plot.height = 6)

p1

In [ ]:
df_plot <- meta.data %>%
    group_by(region, Clusters) %>%
    summarise(counts = n()) %>%
    mutate(cell_proportion = counts / sum(counts))

p1 <- ggplot(df_plot, aes(region, cell_proportion, fill=Clusters)) + 
    geom_bar(stat="identity", position = position_stack(reverse = TRUE)) +
    scale_fill_manual(values = cols) +
    theme_cowplot() +
    xlab("") + ylab("") +
    theme(axis.text.x = element_text(angle = 60, hjust = 1))


options(repr.plot.width = 10, repr.plot.height = 6)

p1

In [ ]:
df_plot <- meta.data %>%
    group_by(patient_group, Clusters) %>%
    summarise(counts = n()) %>%
    mutate(cell_proportion = counts / sum(counts))

p1 <- ggplot(df_plot, aes(patient_group, cell_proportion, fill=Clusters)) + 
    geom_bar(stat="identity", position = position_stack(reverse = TRUE)) +
    scale_fill_manual(values = cols) +
    theme_cowplot() +
    xlab("") + ylab("") +
    theme(axis.text.x = element_text(angle = 60, hjust = 1))


options(repr.plot.width = 6, repr.plot.height = 6)

p1

In [ ]:
saveArchRProject(ArchRProj = proj, 
                 load = FALSE)

In [ ]:
sessionInfo()